In [52]:
import os
import pandas as pd
import re

def masterfile():
    # Paths
    folderPath = "FOLDER"  # Update this
    masterPath = "Master datasheet.xlsx"  # Update this
    outputPath = "master/master.xlsx"  # Update this

    def generateCol(n):
        """ Generate Excel-style column names (A, B, C... AA, AB, etc.). """
        names = []
        for i in range(n):
            col_name = ''
            while i >= 0:
                col_name = chr(i % 26 + 65) + col_name
                i = i // 26 - 1
            names.append(col_name)
        return names

    # Read the master template
    master = pd.read_excel(masterPath)
    masterColnames = master.columns  # Preserve original column names
    n = len(master.columns)
    Colnames = generateCol(n)
    master.columns = Colnames  # Use temporary column names for processing

    # Loop through all folders in the directory
    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)
        if not os.path.isdir(paths):  # Skip files, only process directories
            continue
        
        for filename in os.listdir(paths):
            filePath = os.path.join(paths, filename)

            try:
                # Read the column mapping sheet (handle missing sheets)
                try:
                    Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                except ValueError:
                    print(f"Warning: Sheet '{foldername}' not found in master template. Skipping.")
                    continue

                if not {'From', 'To'}.issubset(Col_directions.columns):
                    print(f"Error: Mapping sheet {foldername} must contain 'From' and 'To' columns.")
                    continue

                # Read the file to process
                fileRead = pd.read_excel(filePath)
                temp_data = []  # Temporary storage for processed rows

                # Breaking measurement columns into M1, M2, M3
                if 'Measurement' in fileRead.columns:
                    fileRead[['m1', 'm2', 'm3']] = fileRead['Measurement'].str.split(' x ', expand=True)
                    fileRead[['m1', 'm2', 'm3']] = fileRead[['m1', 'm2', 'm3']].apply(pd.to_numeric, errors='coerce')

                def extract_dimensions(measurement):
                    match = re.match(r"(\d+\.\d+)-(\d+\.\d+)\*(\d+\.\d+)", str(measurement))
                    return match.groups() if match else (None, None, None)
                print(filename)
                if 'Diameter' in fileRead.columns:
                    fileRead[['m1', 'm2', 'm3']] = fileRead['Diameter'].apply(lambda x: pd.Series(extract_dimensions(x)))
             
                # Assign dynamic column names to the current file
                n = len(fileRead.columns)
                Colnames = generateCol(n)
                fileRead.columns = Colnames

                # Process rows based on column mapping
                for _, row in fileRead.iterrows():
                    if pd.isna(row['A']):  # Stop processing if empty row
                        break

                    new_row = {'A': filename}  # Track file name for reference

                    for _, movement_row in Col_directions.iterrows():
                        from_col = movement_row['From']
                        to_col = movement_row['To']

                        if from_col in row.index and to_col in master.columns:
                            new_row[to_col] = row[from_col]

                    temp_data.append(new_row)  # Add processed row

                # Append processed data to master
                temp_df = pd.DataFrame(temp_data)
                master = pd.concat([master, temp_df], ignore_index=True)

            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue
            
    # Function to calculate 'R' column
    def calculate_R(row):
        if 'P' in row and 'Q' in row and pd.notna(row['P']) and pd.notna(row['Q']):
            return ((100 - row['Q']) / 100) * row['P'] if row['Q'] > 0 else ((100 + row['Q']) / 100) * row['P']
        return None

    # Function to calculate 'S' column
    def calculate_S(row):
        if 'R' in row and 'C' in row and pd.notna(row['R']) and pd.notna(row['C']):
            return round(row['R'] * row['C'])  # Perform calculation
        return None

    # Apply calculations
    if 'P' in master.columns and 'Q' in master.columns:
        master['R'] = master.apply(calculate_R, axis=1)
    if 'R' in master.columns and 'C' in master.columns:
        master['S'] = master.apply(calculate_S, axis=1)

    # Clean column 'I'
    if 'I' in master.columns:
        master['I'] = master['I'].apply(lambda x: None if pd.isna(x) or str(x).strip() == '' else x)

    # Restore original column names
    master.columns = masterColnames

    # Save the final master file
    os.makedirs(os.path.dirname(outputPath), exist_ok=True)  # Ensure directory exists
    master.to_excel(outputPath, index=False)
    print(f"Master file updated and saved at {outputPath}.")



In [53]:
import os
import pandas as pd
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils import column_index_from_string

def hyperlink():
    folderPath = "FOLDER"
    masterPath = "Master datasheet.xlsx"
    outputPath = "target.xlsx"
    targetCol = column_index_from_string('AC')  # Column A for data
    # Column B for filenames
    ReferenceCol = column_index_from_string('AD')
    rowCount = 1

    if os.path.exists("master\\master.xlsx"):
        target = load_workbook("master\\master.xlsx")
        targetSheet = target.active
     # Add headers

    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)

        for filename in os.listdir(paths):
            try:
                filePath = os.path.join(paths, filename)
                Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                source = load_workbook(filePath)
                sourceSheet = source.active

                sourceCol = column_index_from_string(Col_directions['Hfrom'][0])
                RefCol = column_index_from_string(Col_directions['StkRf'][0])

                for count, row in enumerate(sourceSheet.iter_rows(min_row=2), start=2):
                    cell = row[sourceCol - 1]
                    RefCell = row[RefCol-1]
                    targetCell = targetSheet.cell(row=rowCount, column=targetCol)
                    targetCell.value = cell.value
                    targetSheet.cell(row=rowCount,column=ReferenceCol,value=RefCell.value)

                    # Store filename in column B
                    

                    if cell.hyperlink:
                        targetCell.hyperlink = cell.hyperlink
                        targetCell.style = "Hyperlink"

                        rowCount += 1

                print(f"Processed {filename} successfully")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    target.save(outputPath)
    print("Done")






In [55]:
import threading

t1= threading.Thread(target=masterfile)
t2 = threading.Thread(target=hyperlink)

t1.start()
t1.join()
t2.start()
t2.join()

DDPLStock-21-Jan-2025-10-28-57.xlsx


C:\Users\91965\AppData\Local\Temp\ipykernel_20004\1245401959.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master = pd.concat([master, temp_df], ignore_index=True)


FINESTAR_21_01_2025_04_39_52487.xlsx
FINESTAR_21_01_2025_04_43_53IBT.xlsx
Jodhani-Brothers-Stock-21012025-10-15-54.xlsx
Lab_EXL_210125104511.xlsx
Lab_EXL_210125104820.xlsx
Lab_EXL_210125105200.xlsx
Master file updated and saved at master/master.xlsx.
Processed DDPLStock-21-Jan-2025-10-28-57.xlsx successfully
Processed FINESTAR_21_01_2025_04_39_52487.xlsx successfully
Processed FINESTAR_21_01_2025_04_43_53IBT.xlsx successfully
Processed Jodhani-Brothers-Stock-21012025-10-15-54.xlsx successfully
Processed Lab_EXL_210125104511.xlsx successfully
Processed Lab_EXL_210125104820.xlsx successfully
Processed Lab_EXL_210125105200.xlsx successfully
Done


In [ ]:
import os
import pandas as pd
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils import column_index_from_string

def hyperlink():
    folderPath = "FOLDER"
    masterPath = "Master datasheet.xlsx"
    outputPath = "target.xlsx"
    targetCol = column_index_from_string('AC')  # Column A for data
    # Column B for filenames
    ReferenceCol = column_index_from_string('AD')
    rowCount = 1

    if os.path.exists("master\\master.xlsx"):
        target = load_workbook("master\\master.xlsx")
        targetSheet = target.active
     # Add headers

    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)

        for filename in os.listdir(paths):
            try:
                filePath = os.path.join(paths, filename)
                Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                source = load_workbook(filePath)
                sourceSheet = source.active

                sourceCol = column_index_from_string(Col_directions['Hfrom'][0])
                RefCol = column_index_from_string(Col_directions['StkRf'][0])

                for count, row in enumerate(sourceSheet.iter_rows(min_row=1), start=2):
                    cell = row[sourceCol - 1]
                    RefCell = row[RefCol-1]
                    targetCell = targetSheet.cell(row=rowCount, column=targetCol)
                    targetCell.value = cell.value
                    targetSheet.cell(row=rowCount,column=ReferenceCol,value=RefCell.value)

                    # Store filename in column B
                    

                    if cell.hyperlink:
                        targetCell.hyperlink = cell.hyperlink
                        targetCell.style = "Hyperlink"

                        rowCount += 1

                print(f"Processed {filename} successfully")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    target.save(outputPath)
    print("Done")

hyperlink()




Processed DDPLStock-21-Jan-2025-10-28-57.xlsx successfully
Processed FINESTAR_21_01_2025_04_39_52487.xlsx successfully
Processed FINESTAR_21_01_2025_04_43_53IBT.xlsx successfully
Processed Jodhani-Brothers-Stock-21012025-10-15-54.xlsx successfully
Processed Lab_EXL_210125104511.xlsx successfully
Processed Lab_EXL_210125104820.xlsx successfully
Processed Lab_EXL_210125105200.xlsx successfully
Done
